In [1]:
# Load Libaries Needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import category_encoders as ce

In [2]:
X_train=pd.read_csv('X_train.csv',index_col='Rank')
X_test=pd.read_csv('X_test.csv',index_col='Rank')
y_train=pd.read_csv('y_train.csv',index_col='Rank')
y_test=pd.read_csv('y_test.csv',index_col='Rank')

In [3]:
# Cat Boost Encoding 
cat_features= ['Genre','Device']       
target_enc = ce.CatBoostEncoder(cols=cat_features)
target_enc.fit(X_train[cat_features], y_train['Global_Sales'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train_TE = X_train.join(target_enc.transform(X_train[cat_features]).add_suffix('_target'))
test_TE = X_test.join(target_enc.transform(X_test[cat_features]).add_suffix('_target'))

/Users/shailmirpuri/opt/anaconda3/envs/vgsales_predict/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [4]:
X_train=train_TE.drop(columns=cat_features)
X_test=test_TE.drop(columns=cat_features)

In [5]:
y_train=np.reshape(y_train.values,len(y_train))
y_test=np.reshape(y_test.values,len(y_test))

In [6]:
bool_features=['FIFA', 'Mario', 'Call of Duty', 'Grand Theft Auto', 'Pokemon', 'Halo',
       'Wii', 'NBA']
num_features=['Year','NA_Sales As a percentage of Total',
       'EU_Sales As a percentage of Total',
       'JP_Sales As a percentage of Total',
       'Other_Sales As a percentage of Total','Genre_target','Device_target']

In [7]:
# Create Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('normalize',MinMaxScaler())])
bool_transformer=SimpleImputer(strategy='constant')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),('bool',bool_transformer,bool_features)])

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.9 = 0.22

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
def score(model):
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])
    my_pipeline.fit(X_train,y_train)
    pred=my_pipeline.predict(X_val)
    MAE=mean_absolute_error(pred,y_val)
    print(MAE)

In [11]:
from xgboost import XGBRegressor

In [30]:
xgb=XGBRegressor(random_state=1)
score(xgb)

0.33654354540953835


In [35]:
for i in range(100,1001,100):
    xgb=XGBRegressor(n_estimators=i,random_state=1)
    score(xgb)

0.33654354540953835
0.32670909258352226
0.3251101127848568
0.32552496919829843
0.3252835363642757
0.3264922784392933
0.32666490099931184
0.3270007283808876
0.3274039328465341
0.32727535853615725


In [36]:
for i in range(300,401,20):
    xgb=XGBRegressor(n_estimators=i,random_state=1)
    score(xgb)

0.3251101127848568
0.325014061353612
0.3249946553161346
0.32490491376950875
0.3248621706534901
0.32552496919829843


In [40]:
for i in [365,366,373]:
    xgb=XGBRegressor(n_estimators=i,random_state=1)
    score(xgb)

0.32479225216269336
0.32478958106168626
0.32481723395121626


In [43]:
for i in ['gbtree', 'gblinear','dart']:
    xgb= XGBRegressor(random_state=1,booster=i,n_estimators=366)
    score(xgb)

0.32478958106168626
0.5293029650737004
0.32478956096207445


In [44]:
for i in [0.01,0.05,0.10,0.20,0.30,0.40,0.50]:
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=i,n_estimators=366)
    score(xgb)

0.38140600718392265
0.33921942468883204
0.33477309328485205
0.3310265791004442
0.32478958106168626
0.33498222828262464
0.3481255724525835


In [45]:
for i in [0,1,10,100,150,200,500]:
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=i,n_estimators=366)
    score(xgb)

0.32478958106168626
0.3519633320373704
0.37185553888942663
0.4120707264919996
0.43241668701905966
0.427587191934726
0.4946960793473475


In [46]:
for i in range(3,11):
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=i,n_estimators=366)
    score(xgb)

0.3690373064539678
0.34210998643769164
0.3374640834995382
0.32478958106168626
0.3282431797457827
0.33022374830986445
0.32736018033710673
0.32550494685198567


In [47]:
for i in [0,1,10,100]:
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,
                      min_child_weight=i,n_estimators=366)
    score(xgb)

0.32478958106168626
0.32478958106168626
0.365903685411775
0.4074481874358063


In [48]:
for i in range(0,11):
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,
                      min_child_weight=1,max_delta_step=i,n_estimators=366)
    score(xgb)

0.32478958106168626
0.344363062437918
0.34282561629651537
0.33747851641909027
0.3443548468467861
0.3367086231794702
0.3428323244250603
0.3408173110098883
0.3355279991805474
0.34190445465862834
0.34181159295135716


In [49]:
for i in [0.5,0.6,0.7,0.8,0.9,1]:    
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,min_child_weight=1,
                      max_delta_step=0,subsample=i,n_estimators=366)
    score(xgb)

0.3654080731498509
0.35885757618512176
0.3528037990154511
0.3438430081333022
0.3399387776870805
0.32478958106168626


In [50]:
for i in [0.001,0.01,0.1,1,100]:
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,min_child_weight=1,
                      max_delta_step=0,subsample=1,reg_lambda=i,n_estimators=366)
    score(xgb)

0.3370097028513351
0.3360739723637082
0.3306754137952644
0.32478958106168626
0.3507875737294933


In [51]:
for i in ['auto', 'exact', 'approx', 'hist']:
    xgb= XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,min_child_weight=1,
                      max_delta_step=0,subsample=1,reg_lambda=1,tree_method=i,n_estimators=366)
    score(xgb)

0.32478958106168626
0.32478958106168626
0.36109323158608864
0.34415660885219757


In [52]:
# The best hyperparameters 
best_xgb=XGBRegressor(random_state=1,booster='gbtree',eta=0.30,gamma=0,max_depth=6,min_child_weight=1,
                      max_delta_step=0,subsample=1,reg_lambda=1,n_estimators=366,tree_method='auto')